# prompt_builder

> A nano-library for filling the fields of a template, partially and in any order.

This can be handy in situations where you want to build up different variations of a completed template, by filling different fields differently, and this cannot done simply by appending values.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Dict, Set
from string import Formatter


def fields_needed(format_string:str) -> Set[str]:
    "Returns fields needed to complete format_string"
    return set(fname for (_,fname,_,_) in Formatter().parse(format_string) if fname)

`fields_needed` simply tells you which fields are needed to complete the provided string:

In [ ]:
fields_needed("Hello {name}, do you know {mutual_friend}?")

{'mutual_friend', 'name'}

In [ ]:
assert fields_needed("I know you, {friend}") == set({"friend"})

In [ ]:
assert fields_needed("There's nothing to fill here") == set({})

Use `substitute_vals` to fill fields in a template as needed.

In [ ]:
#| export

def substitute_vals(format_string:str,**vals) -> str:
    "Substitutes fields from vals into format_string"
    still_needed = set(fields_needed(format_string)) - set(vals.keys())
    missing_dict = {k:( '{' + k + '}' ) for k in still_needed}
    return format_string.format(**(vals | missing_dict))

You can fill all the values at once:

In [ ]:
substitute_vals("Hello {name}, do you know {mutual_friend}",name="Jack",mutual_friend="Bob")

'Hello Jack, do you know Bob'

Or you can fill just one value, and get back a new format string listing with only the remaining fields:

In [ ]:
substitute_vals("Hello {name}, do you know {mutual_friend}",mutual_friend="Bob")

'Hello {name}, do you know Bob'

The class `Prompt` just wraps up the format string, and the values provided so far, into an instance:

In [ ]:
#| exports

class Prompt:
    def __init__(self,format_string):
      'Initializees a prompt from a format string'
      self.format_string = format_string
      self.values:Dict[str,str] = {}
    def add(self,**new_values):
      "Fill the prompt's fields with new values"
      self.values = self.values | new_values
      return self
    def needed(self) -> Set[str]:
      "Returns the fields not yet filled"
      return set(fields_needed(self.format_string)) - set(self.values.keys())
    def text(self) -> str:
      "Returns the prompt, filled completely or partially"
      return substitute_vals(self.format_string,**self.values)

Then you can put your prompt into an object:

In [ ]:
p = Prompt("""I will provide a transcript. Please review it.

As you review it, identify {topic_count} topics in the transcript, 
and extract {quote_count} verbatim quotes related to the topic. Every 
verbatim quote should be between {min_size} and {max_size} words long.

Then reply the list of topics, providing the quotes for each topic:


```
{transcript}
```

Okay, generate your report on this transcript:
""")

And track the needed fields as you use it.

In [ ]:
p.needed()

{'max_size', 'min_size', 'quote_count', 'topic_count', 'transcript'}

In [ ]:
assert p.needed() == set({'max_size', 'min_size', 'quote_count', 'topic_count', 'transcript'})

You can use method-chaining syntax to add field values incrementally.

In [ ]:
p.add(quote_count='5').add(min_size='100',max_size='500').add(topic_count='1')
assert p.needed() == set({'transcript'})

And feed the output to plain Python string formatting later:

In [ ]:
# returns a new template with only the missing values as named fields
s = p.text() 
# Use plain old string.format to complete that field
print(s.format(transcript="Hi, bob. This interview is done!"))

I will provide a transcript. Please review it.

As you review it, identify 1 topics in the transcript, 
and extract 5 verbatim quotes related to the topic. Every 
verbatim quote should be between 100 and 500 words long.

Then reply the list of topics, providing the quotes for each topic:


```
Hi, bob. This interview is done!
```

Okay, generate your report on this transcript:



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()